# Récupération des données de boursorama

## Scraping de la bourse parisienne

<strong> La variation du chiffre d affaires est récupérée sur 1 an, 3 ans et 5 ans <strong>
Attention, le programme met 5 minutes à tourner

On commence par importer les bibliothèques nécesaires:
1. La biliothèque permettant de récupérer le contenu d'un site internet à partir d'un lien
2. La bibliothèque permettant de trier et réciprérer facilement les données recherchées sur ce contenu
3. La bibliothèque pour debuggage

In [ ]:
import requests
from bs4 import BeautifulSoup

Les données sont enregistrées dans des listes:
* libellé enregistre le nom des sociétés côtées à la bourse parisienne
* var1an enregsitre le nom des sociétés dont on connait la variation de la capitalisation bpursière sur 1 an
* var3an
* var5an
* volume
* capB est la capitalisation boursière
* secteur d'activité 1
* secteur d'activité 2
* secteur d'activité 3

In [10]:
données=[[], [], [], [],[], [], [],[],[]]

Les paramètres changent:
* secteur d'activité, 
* période de varition 1an
* nombre de pages
* bourse (pour les étrangères)
Dans la requête (lien http), seul leur numéro associé importe

In [11]:
secteurs=["Pétrole et gaz", "Matériaux de base", "Industries", "Biens de consommation", "Santé", "Services aux consommateurs", "Télécommunication", "Services aux collectivités", "Sociétés financières", "Technologies", "Banques", ""]
secteursN=["0","1","2","3","4","5","6","7","8","9","835", ""]
périodes=["actuel", "variation sur 1 an", "variation sur 3 ans", "variation sur 5 ans"]
périodesN=["", "5", "11", "12"]

Pour Chaque secteur d'activité, on scrape toutes les cotations de la bourse parisienne. Ensuite on scrape la variation sur 1, 3 puis 5 ans.

In [12]:
for sect in range(len(secteursN)):
    for ipériode in range(len(périodesN)):
        url="https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D="+secteursN[sect]+"&france_filter%5Bvariation%5D=50004&france_filter%5Bperiod%5D="+périodesN[ipériode]+"&france_filter%5Bfilter%5D="
        r=requests.get(url)
        soup=BeautifulSoup(r.text, "html.parser")
        results=soup.find_all("span", attrs={"class":"c-pagination__content"})
        if len(results)==0:
            NPages=2
        else:
            NPages=results[-1].text
            print(url)
        for page in range(1, int(NPages)+1):
            url="https://www.boursorama.com/bourse/actions/palmares/france/page-"+str(page)+"?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D="+secteursN[sect]+"&france_filter%5Bvariation%5D=50004&france_filter%5Bperiod%5D="+périodesN[ipériode]+"&france_filter%5Bfilter%5D="
            r=requests.get(url)
            soup=BeautifulSoup(r.text, "html.parser")
            result1=soup.find("tbody", attrs={'class':"c-table__body"})
            results=result1.find_all("tr", attrs={'class':"c-table__row"})
            if len(results)>0:
                for ligne in range(1, len(results)-1):
                    libellé=results[ligne].find("a").text
                    variation=results[ligne].find("span", attrs={ "class":"c-instrument c-instrument--instant-variation"}).text
                    if not libellé in données[0]:
                        données[0]=données[0]+[libellé]
                        données[4]=données[4]+[results[ligne].find("span", attrs={ "class":"c-instrument c-instrument--totalvolume"}).text]
                        données[5]=données[5]+[(results[ligne].find_all("td", attrs={ "class":"c-table__cell c-table__cell--dotted "})[-1].text).strip()]
                        données[6]=données[6]+[secteurs[sect]]
                        données[7]=données[7]+[""]
                        données[8]=données[8]+[""]
                        if ipériode==0:
                            données[1]=données[1]+[""]
                            données[2]=données[2]+[""]
                            données[3]=données[3]+[""]
                        else:
                            for i in range(1, len(périodesN)):
                                if i==ipériode:
                                    données[i]=données[i]+[variation]
                                else:
                                    données[i]=données[i]+[""]
                    else:
                        ind=données[0].index(libellé)
                        if données[6][ind]!="" and données[6][ind]!=secteurs[sect]:
                            if données[7][ind]!="" and données[7][ind]!=secteurs[sect]:
                                données[8][ind]=secteurs[sect]
                            else:
                                données[7][ind]=secteurs[sect]
                        else:
                            données[6][ind]=secteurs[sect]
                        for i in range(1, len(périodesN)):
                            if ipériode==i:
                                données[i][ind]=variation

https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D=1&france_filter%5Bvariation%5D=50004&france_filter%5Bperiod%5D=11&france_filter%5Bfilter%5D=
https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D=1&france_filter%5Bvariation%5D=50004&france_filter%5Bperiod%5D=12&france_filter%5Bfilter%5D=
https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D=2&france_filter%5Bvariation%5D=50004&france_filter%5Bperiod%5D=&france_filter%5Bfilter%5D=
https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D=2&france_filter%5Bvariation%5D=50004&france_filter%5Bperiod%5D=5&france_filter%5Bfilter%5D=
https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=TOUS&france_filter%5Bsector%5D=2&france_filter%5Bvariation%5D=50004&france_filter%

KeyboardInterrupt: 

On fait à peu près la même chose pour les bourses étrangères

In [ ]:
étranger=[[], [], [], [], [], [], [], [], []]
bourses=["USA, Nyse", "Canada, Toronto", "UK, LSE"]
Npays=["1", "2", "44"]
Nbourse=["101", "205", "440"]

étranger=
* libellé
* var 1 an
* var 3 ans
* var 5 ans
* volume
* capitalisation boursière
* pays 1
* pays 2
* pays 3

In [ ]:
for bourse in range(len(bourses)):
    for ipériode in range(len(périodesN)):
        url="https://www.boursorama.com/bourse/actions/palmares/international/?international_filter%5Bcountry%5D="+Npays[bourse]+"&international_filter%5BindexTrading%5D="+Nbourse[bourse]+"&international_filter%5Bvariation%5D=5004&international_filter%5Bperiod%5D="+périodesN[ipériode]+"&international_filter%5Bfilter%5D="
        r=requests.get(url)
        soup=BeautifulSoup(r.text, "html.parser")
        results=soup.find_all("span", attrs={"class":"c-pagination__content"})
        if len(results)==0:
            NPages=2
        else:
            NPages=results[-1].text
        print(url)
        for page in range(1, int(NPages)+1):
            url="https://www.boursorama.com/bourse/actions/palmares/international/page-"+str(page)+"?international_filter%5Bcountry%5D="+Npays[bourse]+"&international_filter%5BindexTrading%5D="+Nbourse[bourse]+"&international_filter%5Bvariation%5D=5004&international_filter%5Bperiod%5D="+périodesN[ipériode]+"&international_filter%5Bfilter%5D="
            r=requests.get(url)
            soup=BeautifulSoup(r.text, "html.parser")
            result1=soup.find("tbody", attrs={'class':"c-table__body"})
            results=result1.find_all("tr", attrs={'class':"c-table__row"})
            if len(results)>0:
                for ligne in range(len(results)):
                    libellé=results[ligne].find("a").text
                    variation=results[ligne].find("span", attrs={ "class":"c-instrument c-instrument--instant-variation"}).text
                    if not libellé in étranger[0]:
                        étranger[0]=étranger[0]+[libellé]
                        étranger[4]=étranger[4]+[results[ligne].find("span", attrs={ "class":"c-instrument c-instrument--totalvolume"}).text]
                        étranger[5]=étranger[5]+[(results[ligne].find_all("td", attrs={ "class":"c-table__cell c-table__cell--dotted "})[-1].text).strip()]
                        étranger[6]=étranger[6]+[bourses[bourse]]
                        étranger[7]=étranger[7]+[""]
                        étranger[8]=étranger[8]+[""]
                        if ipériode==0:
                            étranger[1]=étranger[1]+[""]
                            étranger[2]=étranger[2]+[""]
                            étranger[3]=étranger[3]+[""]
                        else:
                            for i in range(1, len(périodesN)):
                                if i==ipériode:
                                    étranger[i]=étranger[i]+[variation]
                                else:
                                    étranger[i]=étranger[i]+[""]
                    else:
                        ind=étranger[0].index(libellé)
                        if étranger[6][ind]!="" and étranger[6][ind]!=bourses[bourse]:
                            if étranger[7][ind]!="" and étranger[7][ind]!=bourses[bourse]:
                                étranger[8][ind]=bourses[bourse]
                            else:
                                étranger[7][ind]=bourses[bourse]
                        else:
                            étranger[6][ind]=bourses[bourse]
                        for i in range(1, len(périodesN)):
                            if ipériode==i:
                                étranger[i][ind]=variation

Ensuite on prend le nom des sociétés cotées au SBF120 et CAC40, ou autres cotes étrangères

In [ ]:
sbf=[]
url="https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=1rPPX4&france_filter%5Bsector%5D=&france_filter%5Bvariation%5D=128&france_filter%5Bperiod%5D=2&france_filter%5Bfilter%5D="
r=requests.get(url)
print(url)
soup=BeautifulSoup(r.text, "html.parser")
results=soup.find("div", attrs={"class":"js-table-fixed-column"})
results=soup.find_all("span", attrs={"class":"c-pagination__content"})
if len(results)==0:
    NPages=1
else:
    NPages=results[-1].text
for page in range(1, int(NPages)+1):
    urlsbf="https://www.boursorama.com/bourse/actions/palmares/france/page-"+str(page)+"?france_filter%5Bmarket%5D=1rPPX4&france_filter%5Bsector%5D=&france_filter%5Bvariation%5D=128&france_filter%5Bperiod%5D=2&france_filter%5Bfilter%5D="
    r=requests.get(urlsbf)
    soup=BeautifulSoup(r.text, "html.parser")
    results=soup.find_all("tr", attrs={'class':"c-table__row"})
    for j in range(1, len(results)-1):
        sbf+=[results[j].find("a").contents[0]]
        
cac=[]
url="https://www.boursorama.com/bourse/actions/palmares/france/?france_filter%5Bmarket%5D=1rPCAC&france_filter%5Bsector%5D=&france_filter%5Bvariation%5D=128&france_filter%5Bperiod%5D=11&france_filter%5Bfilter%5D="
r=requests.get(url)
print(url)
soup=BeautifulSoup(r.text, "html.parser")
results=soup.find("div", attrs={"class":"js-table-fixed-column"})
results=soup.find_all("span", attrs={"class":"c-pagination__content"})
if len(results)==0:
    NPages=1
else:
    NPages=results[-1].text
for page in range(0, int(NPages)+1):
    urlcac="https://www.boursorama.com/bourse/actions/palmares/france/page-"+str(page)+"?france_filter%5Bmarket%5D=1rPCAC&france_filter%5Bsector%5D=&france_filter%5Bvariation%5D=128&france_filter%5Bperiod%5D=11&france_filter%5Bfilter%5D="
    r=requests.get(urlcac)
    soup=BeautifulSoup(r.text, "html.parser")
    results=soup.find_all("tr", attrs={'class':"c-table__row"})
    for j in range(1, len(results)-1):
        cac+=[results[j].find("a").contents[0]]
cac=cac[25:]

## Ecriture sur fichier Excel
Dans la première pagesont disposées les informations concernant Paris et dans la deuxième le nom des sociétés par bourse étrangère

In [ ]:
from xlwt import Workbook
 
# création 
book = Workbook()
path = r"T:\_Restricted_BALZAC\Business Dev\SUPERFICHIER\BOURSE\Boursorama.xls"

In [ ]:
# création de la feuille 1
feuil1 = book.add_sheet('bourse parisienne')
 
# ajout des en-têtes
feuil1.write(0,0, "libellé")
feuil1.write(0,1,'variation sur 1 an')
feuil1.write(0,2,'variation sur 3 ans')
feuil1.write(0,3,'variation sur 5 ans')
feuil1.write(0,4,'volume')
feuil1.write(0,5,'capitalisation boursière -M€')
feuil1.write(0,6,'est dans SBF120')
feuil1.write(0,7,'est dans CAC40')
feuil1.write(0,8,"secteur d'activité 1")
feuil1.write(0,9,"secteur d'activité 2")
feuil1.write(0,10,"secteur d'activité 3")

# ajout des valeurs dans la ligne suivante
for i in range(len(données[0])):
    for j in range(5+1):
        feuil1.write(i+1,j, données[j][i])
    if sbf.count(données[0][i])>0:
        feuil1.write(i+1,6,"oui")
    else:
        feuil1.write(i+1,6,"non")
    if cac.count(données[0][i])>0:
        feuil1.write(i+1,7,"oui")
    else:
        feuil1.write(i+1,7,"non")
    feuil1.write(i+1, 8, données[6][i])
    feuil1.write(i+1, 9, données[7][i])
    feuil1.write(i+1, 10, données[8][i])

In [ ]:
# création de la feuille 1
feuil2 = book.add_sheet('bourse étrangère')
 
# ajout des en-têtes
feuil2.write(0,0, "libellé")
feuil2.write(0,1,'variation sur 1 an')
feuil2.write(0,2,'variation sur 3 ans')
feuil2.write(0,3,'variation sur 5 ans')
feuil2.write(0,4,'volume')
feuil2.write(0,5,'capitalisation boursière -M€')
feuil2.write(0,6,"pays 1")
feuil2.write(0,7,"pays 2")
feuil2.write(0,8,"pays 3")

# ajout des valeurs dans la ligne suivante
for i in range(len(étranger[0])):
    for j in range(5+1):
        feuil2.write(i+1,j, étranger[j][i])
    feuil2.write(i+1, 6, étranger[6][i])
    feuil2.write(i+1, 7, étranger[7][i])
    feuil2.write(i+1, 8, étranger[8][i])

In [ ]:
book.save(path)